I might be missing some dependencies, but I think this is all of them:

`pip install drake pytransform3d manipulation`

In [ ]:
import argparse
import json
import multiprocessing
import os
import shutil
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pytransform3d.rotations as pyt_r
from PIL import Image
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    Parser,
    RandomGenerator,
    RigidTransform,
    Role,
    RollPitchYaw,
    Simulator,
    UniformlyRandomRotationMatrix,
)

from manipulation.scenarios import AddRgbdSensor, ycb
from manipulation.utils import ConfigureParser, colorize_labels

import manipulation
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.scenarios import AddShape
from manipulation.utils import RenderDiagram, ConfigureParser
import os

from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    RotationMatrix,
    RollPitchYaw,
    MultibodyPlant,
    Demultiplexer,
    DiscreteContactApproximation,
    ConstantVectorSource,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    JointSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Multiplexer,
    Parser,
    PrismaticJoint,
    SceneGraph,
    SpatialInertia,
    Sphere,
    UnitInertia,
    MeshcatVisualizerParams,
    LoadModelDirectivesFromString,
    RigidTransform,
    ProcessModelDirectives,
    AddDefaultVisualization,
    SimIiwaDriver,
    IiwaControlMode,
    LeafSystem,
    PortSwitch,
    SchunkWsgPositionController
)

from pydrake.geometry import (
    RenderEngineVtkParams,
    MakeRenderEngineVtk,
    LightParameter,
    Rgba,
)
from pydrake.systems.sensors import CameraConfig, ApplyCameraConfig
from pydrake.geometry import GeometrySet, Role

In [ ]:
## CONSTANTS
NUM_IMAGES = 10

# Spawn locations
FOCUS_RANGE = 1.0  # meters, centroid objects spawn at, ~ max distance to an object, centered at origin
SPAWN_RANGE = 0.2  # meters, range around focus to spawn objects, centered at focus

# Camera
CAM_SPAWN_RANGE = 0.2  # meters, centered at origin
CAM_HEIGHT_RANGE = (0.6, 3)  # meters from ground
CAM_ORIENTATION_STD = 0.2  # radians, normal distribution std on roll pitch and yaw

# Lights
LIGHT_SPAWN_RANGE = 2.0  # meters, centered at origin
LIGHT_HEIGHT_RANGE = (0.5, 4)  # meters, min and max
LIGHT_RANGE = (2, 6)  # min and max number of scene lights

# Clutter
CLUTTER_RANGE = (0, 5)  # number of clutter, min and max

# OOIs
OOI_RANGE = (1, 3)  # number of OOIs, min and max
OOI_COLORS = [(1, .75, .3), (.5, 1, .6), (.55, .4, .8), (1, .55, .6)]  # RGB
OOI_COLOR_STD = 0.1  # small noise applied to the selected colors
OOI_SIZE_RANGE = (0.05, 0.1)  # meters, min and max sizes ~ diameter

In [ ]:
# Clear current dataset
if os.path.exists("images"):
    shutil.rmtree("images")
os.makedirs("images")

In [ ]:
world_directives = f"""
directives:
- add_model:
    name: table
    file: file://{os.getcwd()}/models/ground.sdf
- add_weld:
    parent: world
    child: table::base
"""

In [ ]:
clutter_directive = """
directives:
- add_model:
    name: clutter{num}
    file: package://manipulation/hydro/{name}
"""

In [ ]:
box_xml = """
<mujoco model="box{num}">
  <worldbody>
    <body name="box{num}" pos="0 0 1" euler="30 0 45">
      <geom name="box{num}" type="box" size="{size} {size} {size}" rgba="{r} {g} {b} 1"/>
      <joint type="free" name="free"/>
    </body>
  </worldbody>
</mujoco>
"""

sphere_xml = """
<mujoco model="sphere{num}">
  <worldbody>
    <body name="sphere{num}" pos="0 0 1" euler="30 0 45">
      <geom name="sphere{num}" type="sphere" size="{size}" rgba="{r} {g} {b} 1"/>
      <joint type="free" name="free"/>
    </body>
  </worldbody>
</mujoco>
"""

class cone_urdf:

  def format(self, size, r, g, b, num):
    size = 6 + (size - (OOI_SIZE_RANGE[0] + OOI_SIZE_RANGE[1]) / 2)

    return f"""<?xml version='1.0'?>
<sdf version='1.8'>
  <model name='cone_{num}'>
    <link name='link'>
      <visual name='visual'>
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <mesh>
            <uri>file://{os.getcwd()}/models/traffic_cone.obj</uri>
            <scale>{size} {size} {size}</scale>
          </mesh>
        </geometry>
        <material>
          <ambient>{r} {g} {b} 1</ambient>
          <diffuse>{r} {g} {b} 1</diffuse>
        </material>
      </visual>
      <collision name='collision'>
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <mesh>
            <uri>file://{os.getcwd()}/models/traffic_cone.obj</uri>
            <scale>{size} {size} {size}</scale>
          </mesh>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

oois = [(box_xml, ".xml"), (sphere_xml, ".xml"), (cone_urdf(), ".sdf")]

In [ ]:
rng = np.random.default_rng()  # this is for python
generator = RandomGenerator(rng.integers(1000))  # for c++

In [ ]:
def lookat(cam_position: np.ndarray, target_position: np.ndarray) -> np.ndarray:
    forward = target_position - cam_position
    forward = forward / np.linalg.norm(forward)

    right = np.cross(forward, np.array([0, 0, 1]))
    right = right / np.linalg.norm(right)

    up = np.cross(forward, right)
    up = up / np.linalg.norm(up)

    return np.column_stack((right, up, forward))

In [ ]:
def make_random_light():
    intensity = float(10 ** rng.uniform(-1.0, 0.8))
    color = Rgba(
        rng.uniform(0.7, 1.0),
        rng.uniform(0.65, 1.0),
        rng.uniform(0.6, 1.0),
        1.0
    )
    return LightParameter(
        type="point",
        position=[
            rng.uniform(-LIGHT_SPAWN_RANGE, LIGHT_SPAWN_RANGE),
            rng.uniform(-LIGHT_SPAWN_RANGE, LIGHT_SPAWN_RANGE),
            rng.uniform(LIGHT_HEIGHT_RANGE[0], LIGHT_HEIGHT_RANGE[1])],
        intensity=intensity,
        color=color,
        frame="world",
    )

def clip_color(c):
    return max(min(c, 1), 0)

In [ ]:
def get_render_labels(inspector, model_ids):
    all_geom_ids = inspector.GetAllGeometryIds(Role.kPerception)
    render_labels = []
    for geom_id in all_geom_ids:
        frame_id = inspector.GetFrameId(geom_id)
        frame_group = inspector.GetFrameGroup(frame_id)
        if frame_group in model_ids:
            props = inspector.GetProperties(geom_id, Role.kPerception)
            if props is None:
                continue
            if props.HasProperty("label", "id"):
                rl = props.GetProperty("label", "id")
                render_labels.append(int(rl))
    return render_labels

In [ ]:
def image_scene():
    # Pick an (x,y) location to spawn everything around
    focus = (
        rng.uniform(-FOCUS_RANGE, FOCUS_RANGE),
        rng.uniform(-FOCUS_RANGE, FOCUS_RANGE),
    )

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)

    renderer_name = "vtk_renderer_egl"
    vtk_params = RenderEngineVtkParams()
    vtk_params.cast_shadows = True
    vtk_params.force_to_pbr = False

    # Random lights
    vtk_params.lights = [make_random_light() for _ in range(int(rng.integers(LIGHT_RANGE[0], LIGHT_RANGE[1])))]

    renderer = MakeRenderEngineVtk(vtk_params)
    scene_graph.AddRenderer(renderer_name, renderer)

    parser = Parser(plant)
    ConfigureParser(parser)

    directive = LoadModelDirectivesFromString(world_directives)
    ProcessModelDirectives(directive, parser)

    # Spawn clutter in the environment
    num_clutter = rng.integers(CLUTTER_RANGE[0], CLUTTER_RANGE[1])
    for clutter_num in range(num_clutter):
        clutter_name = ycb[rng.integers(len(ycb))]
        directive = clutter_directive.format(num=clutter_num, name=clutter_name)

        parser.AddModelsFromString(directive, ".dmd.yaml")[0]

    # Spawn objects of interest
    num_ooi = rng.integers(OOI_RANGE[0], OOI_RANGE[1])
    ooi_labels = [[], [], []]
    for ooi_num in range(num_ooi):
        ooi_index = rng.integers(len(oois))
        raw_directive, d_type = oois[ooi_index]
        color = OOI_COLORS[rng.integers(len(OOI_COLORS))]
        size = rng.uniform(OOI_SIZE_RANGE[0], OOI_SIZE_RANGE[1])
        directive = raw_directive.format(
            num=ooi_num,
            size=size,
            r=clip_color(color[0] + rng.normal(0, OOI_COLOR_STD)),
            g=clip_color(color[1] + rng.normal(0, OOI_COLOR_STD)),
            b=clip_color(color[2] + rng.normal(0, OOI_COLOR_STD)),
        )

        id = parser.AddModelsFromString(directive, d_type)[0]
        ooi_labels[ooi_index].append(int(id))

    plant.Finalize()

    # Camera is looking at the focus, with orientation noise
    cam_position = np.array(
        [
            rng.uniform(-CAM_SPAWN_RANGE, CAM_SPAWN_RANGE),
            rng.uniform(-CAM_SPAWN_RANGE, CAM_SPAWN_RANGE),
            rng.uniform(CAM_HEIGHT_RANGE[0], CAM_HEIGHT_RANGE[1]),
        ]
    )
    focus_position = np.array([*focus, 0.0])

    R_lookat = lookat(cam_position, focus_position)
    rpy_lookat = pyt_r.euler_from_matrix(
        R_lookat, i=0, j=1, k=2, extrinsic=True
    ) + rng.normal(loc=0, scale=CAM_ORIENTATION_STD, size=3)

    # Spawn camera
    camera = AddRgbdSensor(
        builder,
        scene_graph,
        RigidTransform(RollPitchYaw(rpy_lookat), cam_position),  # type: ignore
        renderer="vtk_renderer_egl",
    )
    camera.set_name("rgbd_sensor")
    builder.ExportOutput(camera.color_image_output_port(), "color_image")
    builder.ExportOutput(camera.label_image_output_port(), "label_image")
    builder.ExportOutput(camera.depth_image_32F_output_port(), "depth_image")

    diagram = builder.Build()

    # Spawn all objects and let them settle
    while True:
        simulator = Simulator(diagram)
        context = simulator.get_mutable_context()
        plant_context = plant.GetMyContextFromRoot(context)

        z = 0.1
        for body_index in plant.GetFloatingBaseBodies():
            tf = RigidTransform(
                UniformlyRandomRotationMatrix(generator),
                [
                    rng.uniform(focus[0] - SPAWN_RANGE, focus[0] + SPAWN_RANGE),
                    rng.uniform(focus[1] - SPAWN_RANGE, focus[1] + SPAWN_RANGE),
                    z,
                ],  # type: ignore
            )
            plant.SetFreeBodyPose(plant_context, plant.get_body(body_index), tf)
            z += 0.1

        try:
            simulator.AdvanceTo(1.0)
            simulator.AdvanceTo(2.0)
            break
        except RuntimeError:
            pass

    # Grab the images from the camera
    color_image = diagram.GetOutputPort("color_image").Eval(context).data
    label_image = diagram.GetOutputPort("label_image").Eval(context).data
    depth_image = diagram.GetOutputPort("depth_image").Eval(context).data

    inspector = scene_graph.model_inspector()

    # Brute force match the labels to their OOI
    output_label_image = np.zeros_like(label_image)
    for label_num, model_ids in enumerate(ooi_labels):
        if len(model_ids) == 0:
            continue
        labels = get_render_labels(inspector, model_ids)
        mask = np.isin(label_image, labels)
        output_label_image += np.where(mask, label_num + 1, 0)

    return color_image, depth_image, output_label_image

In [ ]:
color_image, depth_image, label_image = image_scene()
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(color_image)
plt.axis("off")
plt.subplot(132)
plt.imshow(depth_image, cmap='gray')
plt.axis("off")
plt.subplot(133)
plt.imshow(colorize_labels(label_image))
plt.axis("off")
plt.show()

In [ ]:
# Image.fromarray(color_image.data).save(f"{filename_base}.png")
# np.save(f"{filename_base}_mask", label_image.data)